## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luderitz,NaN,-26.6481,15.1594,63.81,33,13,12.24,few clouds
1,1,Cape Town,ZA,-33.9258,18.4232,60.12,79,0,3.44,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,35.71,87,0,11.50,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,44.26,61,20,10.36,few clouds
4,4,Walvis Bay,NaN,-22.9575,14.5053,52.75,43,0,1.14,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Sao Filipe,CV,14.8961,-24.4956,78.12,76,31,8.10,scattered clouds
7,7,Mahebourg,MU,-20.4081,57.7000,76.24,78,75,11.50,broken clouds
9,9,Bengkulu,ID,-3.8004,102.2655,80.83,78,67,4.83,light rain
10,10,Hilo,US,19.7297,-155.0900,75.25,89,100,9.22,moderate rain
12,12,Chaozhou,CN,23.6651,116.6379,89.46,60,92,6.40,overcast clouds
14,14,Vaini,TO,-21.2000,-175.2000,75.36,88,75,9.22,broken clouds
17,17,Acapulco,MX,16.8634,-99.8901,76.35,88,100,4.25,overcast clouds
21,21,Faanui,PF,-16.4833,-151.7500,78.60,75,64,19.69,broken clouds
25,25,Taoudenni,ML,22.6783,-3.9836,89.15,24,17,7.99,few clouds
26,26,Namatanai,PG,-3.6667,152.4333,82.09,72,90,4.25,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
# preferred_cities_df.count()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna(how='any')
clean_df.count()

City_ID                653
City                   653
Country                653
Lat                    653
Lng                    653
Max Temp               653
Humidity               653
Cloudiness             653
Wind Speed             653
Current Description    653
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sao Filipe,CV,78.12,scattered clouds,14.8961,-24.4956,
1,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000,
2,Bengkulu,ID,80.83,light rain,-3.8004,102.2655,
3,Hilo,US,75.25,moderate rain,19.7297,-155.0900,
4,Chaozhou,CN,89.46,overcast clouds,23.6651,116.6379,
...,...,...,...,...,...,...,...
648,Rudnyy,KZ,75.43,clear sky,52.9594,63.1206,
649,Puerto Cabezas,NI,76.84,broken clouds,14.0351,-83.3888,
650,Langsa,ID,88.27,scattered clouds,4.4683,97.9683,
651,Harrison,US,77.05,clear sky,30.4091,-89.0436,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
#     print(params)

    
# 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
#     print(hotels["results"][0]["name"])

# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7.a Drop the rows where there is no Hotel Name.

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.notnull().sum()

City                   653
Country                653
Max Temp               653
Current Description    653
Lat                    653
Lng                    653
Hotel Name             594
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]

In [11]:
hotel_df.count()

City                   594
Country                594
Max Temp               594
Current Description    594
Lat                    594
Lng                    594
Hotel Name             594
dtype: int64

In [12]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv('WeatherPy_vacation.csv', index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))